In [1]:
import os
import requests
from bs4 import BeautifulSoup, element as el

In [2]:
ROOT_URL = 'http://sonnet91.blogspot.com/'
ROOT_DIR = 'sonnet91.blogspot.com'
ROOT_PATH = os.path.join('../../data', ROOT_DIR)
if not os.path.exists(ROOT_PATH):
    os.mkdir(ROOT_PATH)

In [3]:
index_html = requests.get(ROOT_URL).text
index_soup = BeautifulSoup(index_html)
book_div = index_soup.find_all('div', attrs={'class': 'categories'})
assert len(book_div) == 1
book_anchors = book_div[0].find_all('a')
book_anchor: el.Tag
for book_anchor in book_anchors:
    print(book_anchor.text.strip(), book_anchor.get('href'))

রুপসী বাংলা http://sonnet91.blogspot.com/search/label/%E0%A6%B0%E0%A7%81%E0%A6%AA%E0%A6%B8%E0%A7%80%20%E0%A6%AC%E0%A6%BE%E0%A6%82%E0%A6%B2%E0%A6%BE
অপ্রকাশিত http://sonnet91.blogspot.com/search/label/%E0%A6%85%E0%A6%AA%E0%A7%8D%E0%A6%B0%E0%A6%95%E0%A6%BE%E0%A6%B6%E0%A6%BF%E0%A6%A4
সাতটি তারার তিমির http://sonnet91.blogspot.com/search/label/%E0%A6%B8%E0%A6%BE%E0%A6%A4%E0%A6%9F%E0%A6%BF%20%E0%A6%A4%E0%A6%BE%E0%A6%B0%E0%A6%BE%E0%A6%B0%20%E0%A6%A4%E0%A6%BF%E0%A6%AE%E0%A6%BF%E0%A6%B0
বনলতা সেন http://sonnet91.blogspot.com/search/label/%E0%A6%AC%E0%A6%A8%E0%A6%B2%E0%A6%A4%E0%A6%BE%20%E0%A6%B8%E0%A7%87%E0%A6%A8
ঝরা পালক http://sonnet91.blogspot.com/search/label/%E0%A6%9D%E0%A6%B0%E0%A6%BE%20%E0%A6%AA%E0%A6%BE%E0%A6%B2%E0%A6%95
ধূসর পাণ্ডুলিপি http://sonnet91.blogspot.com/search/label/%E0%A6%A7%E0%A7%82%E0%A6%B8%E0%A6%B0%20%E0%A6%AA%E0%A6%BE%E0%A6%A3%E0%A7%8D%E0%A6%A1%E0%A7%81%E0%A6%B2%E0%A6%BF%E0%A6%AA%E0%A6%BF
বেলা অবেলা কালবেলা http://sonnet91.blogspot.com/search/label/%E0%A6%AC%E0%A7%87%E0

In [4]:
import json
import re

def scrap_book(url: str, name: str):
    content_path = os.path.join(ROOT_PATH, name)
    if not os.path.exists(content_path):
        os.mkdir(content_path)

    while True:
        book_html = requests.get(url).text
        book_soup = BeautifulSoup(book_html)

        poems = book_soup.select('.post.hentry')
        poem: el.Tag
        for poem in poems:
            h3 = poem.find_all('h3')
            assert len(h3) == 1, len(h3)
            title = h3[0].text.strip()

            p = poem.select('div.post-inner p')
            assert len(p) == 1, f'{len(p)}: {title}'
            content = p[0].get_text(separator='\n').strip()

            data_obj = {
                'title': title, 'author': 'জীবনানন্দ দাশ', 'url': url, 'content': content
            }

            filepath = os.path.join(content_path, re.sub("[^\\u0980-\\u09FF]", "_", title) + '.json')
            with open(filepath, 'w', encoding='utf-8') as file:
                json.dump(data_obj, file, ensure_ascii=False)

        next_page_anchor = book_soup.find_all('a', attrs={'class': 'blog-pager-older-link'})
        assert len(next_page_anchor) <= 1, f'{len(next_page_anchor)}: {url}'

        if len(next_page_anchor) == 0:
            break

        url = next_page_anchor[0].get('href')

In [5]:
from time import sleep

for book_anchor in book_anchors:
    scrap_book(book_anchor.get('href'), book_anchor.text.strip())
    print(f'Scrapped: {book_anchor.text.strip()}')

    sleep(10)

Scrapped: রুপসী বাংলা
Scrapped: অপ্রকাশিত
Scrapped: সাতটি তারার তিমির
Scrapped: বনলতা সেন
Scrapped: ঝরা পালক
Scrapped: ধূসর পাণ্ডুলিপি
Scrapped: বেলা অবেলা কালবেলা
Scrapped: মহাপৃথিবী
Scrapped: শ্রেষ্ঠ কবিতা


In [6]:
print(f'Total {sum(len(files) for *_, files in os.walk(ROOT_PATH))} entries scrapped')

Total 260 entries scrapped
